# Financial Risk Management with ~R~ Python

This notebook is a "translation" of the course "Financial Risk Management with R" taught by Prof. David Hsieh, on Coursera.

Here I will try to re-do all the exercises shown in class, but i will implement the code in python. My focus will be to get the most similar numeric solution possible, of course, this will not always be possible, due to differences in algorithm implementation.

----------------------------------------------------------------------------------------------------------------------------

## Getting Data and Cleaning Data

In [1]:
#Importing the necessary libraries

from fredapi import Fred
import pandas as pd
import numpy as np
from datetime import timedelta
import scipy
import plotly.express as px

In [2]:
#Getting the API KEY and creating the FRED object

with open("../FRED_API_KEY.txt") as txt:
    API_KEY = txt.read()

fred = Fred(api_key=API_KEY)

In [3]:
#Searching on the FRED database for series called "Wilshire 5000"

wilshire_search = fred.search('Wilshire 5000', limit=5)
wilshire_search.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,notes
series id,,,,,,,,,,,,,,,
WILL5000INDFC,WILL5000INDFC,2023-02-23,2023-02-23,Wilshire 5000 Total Market Full Cap Index,1970-12-31,2023-02-22,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-23 07:03:04-06:00,67,The observations for the Wilshire 5000 Total M...
WILL5000PRFC,WILL5000PRFC,2023-02-23,2023-02-23,Wilshire 5000 Full Cap Price Index,1970-12-31,2023-02-22,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-23 07:03:05-06:00,56,The observations for the Wilshire 5000 Full Ca...
WILL5000PR,WILL5000PR,2023-02-23,2023-02-23,Wilshire 5000 Price Index,1970-12-31,2023-02-22,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-23 07:02:04-06:00,54,The observations for the Wilshire 5000 Price I...
WILL5000IND,WILL5000IND,2023-02-23,2023-02-23,Wilshire 5000 Total Market Index,1970-12-31,2023-02-22,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-02-23 07:03:06-06:00,47,The observations for the Wilshire 5000 Total M...


In [4]:
#Getting the data from the Fred database, from 1980 to 2017 of the series "WILL5000IND"

start_date = "1979-12-31"
end_date = "2017-12-31"
will5000 = fred.get_series(series_id='WILL5000IND', observation_start=start_date, observation_end=end_date)
will5000 = will5000.to_frame()

In [5]:
#Organizing column names and cleaning the table

will5000 = will5000.reset_index()
will5000 = will5000.rename(columns={"index": "date", 0: "TR"})

na_s = will5000.TR.isna().sum()
print(f"Number of NA's: {na_s}\n")
will5000 = will5000.dropna()

will5000.head(3)

Number of NA's: 331



,date,TR
0,1979-12-31,1.90
2,1980-01-02,1.86
4,1980-01-04,1.88


In [6]:
#Creating columns to display the return
#log_ret = Continuous compounding
#dis_ret = Discrete compounding

will5000["log_ret"] = np.log(will5000.TR/will5000.TR.shift(1))
will5000["dis_ret"] = will5000.TR.pct_change()
#will5000["dis_ret"] = np.exp(will5000.log_ret)-1

will5000 = will5000.dropna()

In [7]:
print(will5000.head(3))
print("\n###########################################\n")
print(will5000.tail(3))

        date    TR   log_ret   dis_ret
2 1980-01-02  1.86 -0.021277 -0.021053
4 1980-01-04  1.88  0.010695  0.010753
5 1980-01-07  1.89  0.005305  0.005319

###########################################

           date      TR   log_ret   dis_ret
9912 2017-12-27  124.04  0.000807  0.000807
9913 2017-12-28  124.33  0.002335  0.002338
9914 2017-12-29  123.67 -0.005323 -0.005308


In [8]:
#Creating new tables, grouping the return on: a weekly, monthly, quarterly, and annual basis
#In R this would be translated to "apply.weekly(will5000$log_ret, sum)"
#Due to being a different implementation of the algorithm, there are some minor differences in the dates

df = will5000.copy()

df_w = df.groupby([pd.Grouper(key="date", freq="W")])["log_ret"].sum().to_frame().reset_index()
df_w["date"] = pd.to_datetime(df_w.date) - timedelta(days=2)
df_w["dis_ret"] = np.exp(df_w.log_ret)-1
df_w = df_w.rename(columns={"log_ret": "log_ret_w", "dis_ret": "dis_ret_w"})

df_m = df.groupby([pd.Grouper(key="date", freq="M")])["log_ret"].sum().to_frame().reset_index()
df_m["dis_ret"] = np.exp(df_m.log_ret)-1
df_m = df_m.rename(columns={"log_ret": "log_ret_m", "dis_ret": "dis_ret_m"})

df_q = df.groupby([pd.Grouper(key="date", freq="Q")])["log_ret"].sum().to_frame().reset_index()
df_q["dis_ret"] = np.exp(df_q.log_ret)-1
df_q = df_q.rename(columns={"log_ret": "log_ret_q", "dis_ret": "dis_ret_q"})

df_y = df.groupby([pd.Grouper(key="date", freq="Y")])["log_ret"].sum().to_frame().reset_index()
df_y["dis_ret"] = np.exp(df_y.log_ret)-1
df_y = df_y.rename(columns={"log_ret": "log_ret_y", "dis_ret": "dis_ret_y"})


In [9]:
#Merging the data

t_return = pd.merge(df, df_w, how="left", on="date")
t_return = pd.merge(t_return, df_m, how="left", on="date")
t_return = pd.merge(t_return, df_q, how="left", on="date")
t_return = pd.merge(t_return, df_y, how="left", on="date")

t_return = t_return.drop(["dis_ret", "dis_ret_w", "dis_ret_m", "dis_ret_q", "dis_ret_y"], axis=1)

t_return.head(3)

,date,TR,log_ret,log_ret_w,log_ret_m,log_ret_q,log_ret_y
0,1980-01-02,1.86,-0.021277,NaN,NaN,NaN,NaN
1,1980-01-04,1.88,0.010695,-0.010582,NaN,NaN,NaN
2,1980-01-07,1.89,0.005305,NaN,NaN,NaN,NaN


In [13]:
#Ploting quarterly return

fig_q = px.scatter(y = t_return.log_ret_q, x = t_return.date, title="Quarterly Continuous Return")
fig_q.update_xaxes(title_text="Year")
fig_q.update_yaxes(title_text="Return")
fig_q.show()

----------------------------------------------------------------------------------------------------------------------------

## Calculating the Value at Risk and the Expected Shortfall

### *Suposing Normal Distribution


#### Remember
R&emsp;&emsp;&emsp;&emsp;&emsp;Python&emsp;&emsp;&emsp;Name<br />
dnorm()&emsp;->&emsp;df()&emsp;->&emsp;&emsp;Probability density function<br />
pnorm()&emsp;->&emsp;cdf()&emsp;->&emsp;Cumulative density function<br />
qnorm()&emsp;->&emsp;ppf()&emsp;->&emsp;Percentile point function (CDF inverse)<br />

In [14]:
#Calculating the Mean and the SD

mu = will5000["log_ret"].mean() 
sig = will5000["log_ret"].std() 
print(f"Mean: \t\t\t {round(mu, 4)}")
print(f"Standard Deviation: \t {round(sig, 4)}")

Mean: 			 0.0004
Standard Deviation: 	 0.0107


In [15]:
#Calculating the Value at risk

alpha = 0.05

value_at_risk = scipy.stats.norm.ppf(alpha, loc=mu, scale=sig)

print(f"Alpha: \t\t  {alpha}")
print(f"Value at Risk: \t {round(value_at_risk, 4)}")
print(f"Using VaR: \t {round(1000 * (np.exp(value_at_risk)-1), 1)} M loss on a single day for a 1B portfolio.")

Alpha: 		  0.05
Value at Risk: 	 -0.0172
Using VaR: 	 -17.1 M loss on a single day for a 1B portfolio.


In [16]:
#Calculating the Expected Shorfall

ES = mu - sig * scipy.stats.norm.pdf(scipy.stats.norm.ppf(alpha))/alpha

print(f"Alpha: \t\t\t  {alpha}")
print(f"Expected Shortfall: \t {round(ES, 4)}")
print(f"Using ES: \t\t {round(1000 * (np.exp(ES)-1), 1)} M loss on a single day for a 1B portfolio.")

Alpha: 			  0.05
Expected Shortfall: 	 -0.0217
Using ES: 		 -21.4 M loss on a single day for a 1B portfolio.


### * Now Using Simulations
